In [2]:
# Instruções basiconas de sempre.

# Lidar com dataframes.
import numpy as np
import pandas as pd

# Lidar com gráficos.
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from plotting import (multiple_histograms_plot,
                      bar_plot_with_categorical,
                      plot_confusion_matrix,
                      plot_confusion_matrix_2,
                      plot_roc)

# Lidar com preparação de dados.
from data_prep import data_prep as dp # Eu que fiz esse modulinho ("uuuuuuuuuma bosts!").
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (StandardScaler
                                   , MinMaxScaler)
from sklearn.model_selection import (train_test_split
                                     , cross_val_score
                                     , StratifiedKFold)

# Lidar com validação de modelos.
from sklearn.metrics import (confusion_matrix
                             , accuracy_score
                             , classification_report)

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [3]:
#Bibliotecas necessária para conexão com banco de dados MySQL.

import mysql.connector

# Criando uma conexão com o banco e montando um DataFrame
Seguindo as instruções em uma questão do [W3School](https://www.w3schools.com/python/python_mysql_getstarted.asp), temos:

In [29]:
mydb = mysql.connector.connect(
    host="localhost"
    , user="root"
    , passwd=""
    , database="projeto_gastos_parlamentares"
)

mycursor = mydb.cursor()

mycursor.execute("""
SELECT *
FROM tb_gastos_parlamentares_2019
LIMIT 150000;
"""
                 , multi=False)

df_gastos_2019 = mycursor.fetchall()

In [30]:
df_gastos_2019 = pd.DataFrame(df_gastos_2019)

df_gastos_2019.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,2019,7,,FELIPE CARRERAS,PSB,PE,TELEFONIA,00000000000006,RAMAL,102,0,102
1,2019,7,,NORMA AYUB,DEM,ES,TELEFONIA,00000000000006,RAMAL,131,0,131
2,2019,7,,FAUSTO PINATO,PP,SP,TELEFONIA,00000000000006,RAMAL,23,0,23
3,2019,7,,IRACEMA PORTELLA,PP,PI,TELEFONIA,00000000000006,RAMAL,110,0,110
4,2019,7,,FÁBIO HENRIQUE,PDT,SE,TELEFONIA,00000000000006,RAMAL,169,0,169


In [31]:
df_gastos_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145823 entries, 0 to 145822
Data columns (total 12 columns):
0     145823 non-null int64
1     145823 non-null int64
2     145823 non-null object
3     145823 non-null object
4     145823 non-null object
5     145823 non-null object
6     145823 non-null object
7     145823 non-null object
8     145823 non-null object
9     145823 non-null object
10    145823 non-null object
11    145823 non-null object
dtypes: int64(2), object(10)
memory usage: 7.8+ MB


In [33]:
dp.serieNulos(df_gastos_2019,0)

(Series([], dtype: float64),
 '-> 0 atributos/features/campos possuem mais de 0% de valores nulos.')

In [35]:
dp.cardinalidade(df_gastos_2019)

,Atributo,Cardinalidade,Valores
4,6,18,"[TELEFONIA, SERVIÇOS POSTAIS, COMBUSTÍVEIS E LUBRIFICANTES., HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL., CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS., FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR, DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR., MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR, SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO, LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES, PASSAGENS AÉREAS, SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPECIALIZADA., PASSAGENS TERRESTRES, MARÍTIMAS OU FLUVIAIS, LOCAÇÃO OU FRETAMENTO DE AERONAVES, PARTICIPAÇÃO EM CURSO, PALESTRA OU EVENTO SIMILAR, ASSINATURA DE PUBLICAÇÕES, Emissão Bilhete Aéreo, LOCAÇÃO OU FRETAMENTO DE EMBARCAÇÕES]"
3,5,28,"[PE, ES, SP, PI, SE, BA, PR, RS, MG, SC, RJ, AC, PB, PA, AM, MA, CE, GO, MS, RO, RN, TO, MT, RR, AL, DF, AP, NA]"
2,4,35,"[PSB, DEM, PP, PDT, PODE, PT, PRB, CIDADANIA, PSOL, MDB, PCdoB, PL, PTB, PROS, PSDB, PSC, PSD, PSL, SOLIDARIEDADE, NOVO, AVANTE, PATRIOTA, PV, S.PART., REDE, PMN, PHS, PPS, PR, PATRI, PPL, LIDMIN, SDD, LID.GOV-CD, REPUBLICANOS]"
8,10,520,"[0, 2, 8, 4, 9, 1, 178, 6, 5, 57, 12, 24, 85, 20, 30, 62, 86, 3, 49, 236, 59, 16, 139, 13, 31, 15, 7787, 119, 74, 77, 19, 80, 109, 125, 120, 36, 101, 250, 54, 46, 11, 10, 379, 47, 58, 67, 240, 52, 60, 207, 14, 903, 28, 149, 378, 41, 194, 25, 113, 56, 70, 151, 35, 115, 885, 39, 42, 76, 143, 350, 89, 390, 581, 51, 22, 66, 2037, 34, 2960, 220, 99, 75, 165, 123, 32, 176, 7, 150, 40, 1240, 53, 27, 223, 224, 17, 68, 18, 44, 190, 170, ...]"
1,3,815,"[FELIPE CARRERAS, NORMA AYUB, FAUSTO PINATO, IRACEMA PORTELLA, FÁBIO HENRIQUE, BACELAR, ARLINDO CHINAGLIA, CELSO RUSSOMANNO, GUSTAVO FRUET, RUBENS BUENO, HENRIQUE FONTANA, POMPEO DE MATTOS, IVAN VALENTE, JÚLIO DELGADO, ANGELA AMIN, RUI FALCÃO, BENEDITA DA SILVA, HERMES PARCIANELLO, RICARDO BARROS, RENILDO CALHEIROS, DARCÍSIO PERONDI, PERPÉTUA ALMEIDA, WELLINGTON ROBERTO, WILSON SANTIAGO, BOSCO COSTA, ALICE PORTUGAL, DANIEL ALMEIDA, ELCIONE BARBALHO, JOSÉ PRIANTE, ÁTILA LINS, LEONARDO MONTEIRO, MÁRIO HERINGER, ODAIR CUNHA, PATRUS ANANIAS, REGINALDO LOPES, GASTÃO VIEIRA, ANÍBAL GOMES, CARLOS SAMPAIO, GILBERTO NASCIMENTO, JEFFERSON CAMPOS, VICENTINHO, LEÔNIDAS CRISTINO, ROBERTO PESSOA, JÚLIO CESAR, PAES LANDIM, NILSON PINTO, SILAS CÂMARA, JOÃO CAMPOS, RUBENS OTONI, GERALDO RESENDE, VANDER LOUBET, GIACOBO, MARIA DO ROSÁRIO, ONYX LORENZONI, PAULO PIMENTA, GONZAGA PATRIOTA, WOLNEY QUEIROZ, ÁTILA LIRA, DAMIÃO FELICIANO, ANDRÉ DE PAULA, LUCIANO BIVAR, CLAUDIO CAJADO, JOSÉ ROCHA, NELSON PELLEGRINO, PAULO MAGALHÃES, LINCOLN PORTELA, AÉCIO NEVES, EDUARDO BARBOSA, RODRIGO MAIA, MAURO LOPES, LUIZA ERUNDINA, JANDIRA FEGHALI, MAURÍCIO DZIEDRICKI, MARCELO FREIXO, MÁRCIO JERRY, DRA. SORAYA MANATO, PEDRO AUGUSTO BEZERRA, SANTINI, LEUR LOMANTO JÚNIOR, EDUARDO BOLSONARO, STEFANO AGUIAR, CACÁ LEÃO, LAFAYETTE DE ANDRADA, GLEISI HOFFMANN, JAQUELINE CASSOL, BENES LEOCÁDIO, JOÃO MARCELO SOUZA, GELSON AZEVEDO, DARCI DE MATOS, ADRIANO DO BALDY, PEDRO PAULO, PEDRO LUCAS FERNANDES, ENIO VERRI, ANDRÉ FIGUEIREDO, MARCELO MORAES, PAULO RAMOS, EDMILSON RODRIGUES, CRISTIANO VALE, AFONSO HAMM, VICENTINHO JÚNIOR, ...]"
7,9,5317,"[102, 131, 23, 110, 169, 31, 112, 37, 17, 53, 82, 119, 57, 60, 78, 93, 58, 33, 94, 32, 154, 27, 136, 64, 59, 30, 26, 87, 172, 153, 42, 98, 76, 180, 251, 25, 11, 63, 132, 70, 198, 71, 34, 22, 134, 97, 227, 1, 185, 167, 2, 9, 36, 221, 130, 75, 40, 44, 77, 397, 109, 139, 8, 50, 48, 125, 108, 231, 49, 111, 46, 67, 41, 107, 15, 81, 38, 140, 4, 128, 127, 292, 145, 20, 45, 166, 61, 43, 158, 122, 52, 213, 372, 157, 80, 405, 151, 88, 72, 62, ...]"
9,11,5334,"[102, 131, 23, 110, 169, 31, 112, 37, 17, 53, 82, 119, 57, 60, 78, 93, 58, 33, 94, 32, 154, 27, 136, 64, 59, 30, 26, 87, 172, 153, 42, 98, 76, 180, 251, 25, 11, 63, 132, 70, 198, 71, 34, 22, 134, 97, 227, 1, 185, 167, 2, 9, 36, 221, 130, 75, 40, 44, 77, 397, 109, 139, 8, 50, 48, 125, 108, 231, 49, 111, 46, 67, 41, 107, 15, 81, 38, 140, 4, 128, 127, 292, 145, 20, 45, 166, 61,

**Análise preliminar:**

1. Os atributos / fatores / variáveis independentes que possuem cardinalidade igual a um não oferecem nenhum ganho de informação. Atributos 
**Ação:** Remover esses atributos.

2. Os atributos vlrdocumento e vlrliquido estão como objects, mas são floats.
**Ação:** Converter esses atributos para float.

In [ ]:
# Removendo os atributos com cardinalidade <= 1:

lista_atributos_para_remover = list(dp.cardinalidade(df_gastos_parlamentares)[dp.cardinalidade(df_gastos_parlamentares)["Cardinalidade"] <= 1]["Atributo"].values)

df_gastos_parlamentares.drop(lista_atributos_para_remover, axis=1, inplace=True)

In [ ]:
df_gastos_parlamentares.vlrdocumento = pd.to_numeric(df_gastos_parlamentares.vlrdocumento)
df_gastos_parlamentares.vlrliquido = pd.to_numeric(df_gastos_parlamentares.vlrliquido)

df_gastos_parlamentares.info()

In [ ]:
dp.cardinalidade(df_gastos_parlamentares)

In [ ]:
df_gastos_parlamentares.describe().T

In [ ]:
df_gastos_parlamentares.groupby(["sguf", "sgpartido"], axis=0)["vlrliquido"].mean()